In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma (
    embedding_function=embedding_function,
    collection_name='split_1_300_344',
    persist_directory='./split_1_300_344'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [3]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str

graph_builder = StateGraph(AgentState)

In [5]:
# Node는 2가지가 필요
# 1. 문서를 가져오는 retrieve
# 2. 답변을 생성하는 generate

def retrieve(state: AgentState):
    query = state['query']
    docs = retriever.invoke(query)
    return {'context': docs}

In [ ]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
from langchain import hub
# prompt는 genereate, rewrite, 문서 관련성 등 프롬포트가 필요하다.
# 따라서 prompt라는 이름으로는 중복 변수 이름이 될 수 있다.
generate_prompt = hub.pull("rlm/rag-prompt")

def generate(state: AgentState):
    context = state['context']
    query = state['query']
    rag_chain = generate_prompt | llm
    response = rag_chain.invoke({'question': query, 'context': context})
    response = llm.invoke(query, context)
    return {'answer': response}

/root/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from langchain import hub
doc_relevance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relenvance(state: AgentState):
    query = state['query']
    context = state['context']
    doc_relevance_chain = doc_relevance_prompt | llm
    response = doc_relevance_chain.invoke({'question': query, 'documents': context})
    if response['Score'] == 1:
        return 'generate'
    return 'rewrite'

/root/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
query = "Bean Factory와 Application Context의 차이는?"
# Bean Factory와 Application Context의 차이" → "FactoryBean과 Application Context의 차이

In [ ]:
from langchain_core.prompts import PromptTemplate

dictionary = ['사용자와 관련된 표현 -> 클라이언트']
rewrite_prompt = PromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
사전: {dictionary}
질문: {{query}}
""")

# 입력된 쿼리(query)를 변형(rewrite)하여 새로운 쿼리를 생성하는 것
def rewrite(state: AgentState):
    query = state['query']
    rewrite_chain = rewrite_prompt | llm
    response = rewrite_chain.invoke({'query': query})
    return {'query': response}